# Unit 12 - Tales from the Crypto

---


## 1. Sentiment Analysis

Use the [newsapi](https://newsapi.org/) to pull the latest news articles for Bitcoin and Ethereum and create a DataFrame of sentiment scores for each coin.

Use descriptive statistics to answer the following questions:
1. Which coin had the highest mean positive score?
2. Which coin had the highest negative score?
3. Which coin had the highest positive score?

In [1]:
# Initial imports
import os
import pandas as pd
from dotenv import load_dotenv
import nltk as nltk
nltk.download('vader_lexicon')
from nltk.sentiment.vader import SentimentIntensityAnalyzer
analyzer = SentimentIntensityAnalyzer()

%matplotlib inline

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\Drodr\AppData\Roaming\nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


In [2]:
# Read your api key environment variable
# YOUR CODE HERE!

api_key = os.getenv("NEWS_API")

In [3]:
# Create a newsapi client
# YOUR CODE HERE!

newsapi = NewsApiClient(api_key=api_key)

In [4]:
# Fetch the Bitcoin news articles
# YOUR CODE HERE!

btc_headlines = newsapi.get_everything(q="bitcoint", language="en", sort_by="relevancy")
print(f"Total number of Bitcoin articles: {btc_headlines['totalResults']}")

In [5]:
# Fetch the Ethereum news articles
# YOUR CODE HERE!

eth_headlines = newsapi.get_everything(q="ethereum", language="en", sort_by="relevancy")
print(f"Total number of Ethereum articles: {eth_headlines['totalResults']}")

In [1]:
# Create the Bitcoin sentiment scores DataFrame
# YOUR CODE HERE!

btc_sentiments = []

    for article in btc_headlines["articles"]:
        try:
            text = article["content"]
            date = article["publishAt"][:10]
            sentiment = analyzer.polarity_scores(text)
            compound = sentiment["compound"]
            pos = sentiment["pos"]
            neu = sentiment["neu"]
            neg = sentiment["neg"]

            btc_sentiments.append({
                "text": text,
                "date": date,
                "compound": compound,
                "positive": pos,
                "neutral": neu,
                "negative": neg
            })
        
        except AttributeError:
            pass

btc_df = pd.DataFrame(btc_sentiments)

##JR set up columns with text last for viewability

new_columns = ["date", "compound", "positive", "neutral", "negative", "text"]
btc_df = btc_df[new_columns]

btc_df.head()


IndentationError: unexpected indent (376160726.py, line 6)

In [7]:
# Create the Ethereum sentiment scores DataFrame
# YOUR CODE HERE!

eth_sentiments = []

for article in eth_headlines["articles"]:
        try:
            text = article["content"]
            date = article["publishAt"][:10]
            sentiment = analyzer.polarity_scores(text)
            compound = sentiment["compound"]
            pos = sentiment["pos"]
            neu = sentiment["neu"]
            neg = sentiment["neg"]

            eth_sentiments.append({
                "text": text,
                "date": date,
                "compound": compound,
                "positive": pos,
                "neutral": neu,
                "negative": neg
            })
        
        except AttributeError:
            pass

eth_df = pd.DataFrame(eth_sentiments)

##JR set up columns with text last for viewability

new_columns = ["date", "compound", "positive", "neutral", "negative", "text"]
eth_df = eth_df[new_columns]

eth_df.head()

In [8]:
# Describe the Bitcoin Sentiment
# YOUR CODE HERE!

btc.df.describe()


In [9]:
# Describe the Ethereum Sentiment
# YOUR CODE HERE!

eth.df.describe()

### Questions:

Q: Which coin had the highest mean positive score?

A: BTC at .048 (ETH was only .041)

Q: Which coin had the highest compound score?

A: ETH at .69 (vs BTC at .52) MAX

Q. Which coin had the highest positive score?

A: ETH at .178 (vs BTC at .164)

---

## 2. Natural Language Processing
---
###   Tokenizer

In this section, you will use NLTK and Python to tokenize the text for each coin. Be sure to:
1. Lowercase each word.
2. Remove Punctuation.
3. Remove Stopwords.

In [10]:
from nltk.tokenize import word_tokenize, sent_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer, PorterStemmer
from string import punctuation
import re

In [11]:
# Instantiate the lemmatizer
# YOUR CODE HERE!

lemmatizer = WordNetLemmatizer()

# Create a list of stopwords
# YOUR CODE HERE!

sw = set(stopwords.words('english'))

# Expand the default stopwords list if necessary
# YOUR CODE HERE!
##JR Review here
sw_addon = {'ha', 'charsnow', 'charsthe', 'ano', 'depp'}

In [12]:
# Complete the tokenizer function
def tokenizer(text):
    """Tokenizes text."""
    sw = set(stopwords.words('english'))
    regex = re.compile("[^a-zA-Z]")
    re_clean = regex.sub('', text)
    words = word_tokenize(re_clean)
    lem = [lemmatizer.lemmatize(word) for word in words]
    tokens = [word.lower() for word in words if word.lower() not in sw.union(sw_addon)]
    
    # Remove the punctuation from text
    # Create a tokenized list of the words
    # Lemmatize words into root words
    # Convert the words to lowercase
    # Remove the stop words
    
    return tokens

In [13]:
# Create a new tokens column for Bitcoin
# YOUR CODE HERE!

btc_df["tokens"] = btc_df["text"].apply(tokenizer)
btc_df.head()

In [14]:
# Create a new tokens column for Ethereum
# YOUR CODE HERE!

eth_df["tokens"] = eth_df["text"].apply(tokenizer)
eth_df.head()

---

### NGrams and Frequency Analysis

In this section you will look at the ngrams and word frequency for each coin. 

1. Use NLTK to produce the n-grams for N = 2. 
2. List the top 10 words for each coin. 

In [15]:
from collections import Counter
from nltk import ngrams

In [16]:
# Generate the Bitcoin N-grams where N=2
# YOUR CODE HERE!
tokens = []
for i in range(btc_df.shape[0]):
    tokens += btc_df.iloc[i]["text"]

##JR REVIEW > may try different approach here?**
##JR needs to be a string i think:)
btc_tokens = tokenizer(btc_df["text"].str.cat()) 
btc_counts = Counter(ngrams(btc_tokens, n=2))
btc_counts


In [17]:
# Generate the Ethereum N-grams where N=2
# # YOUR CODE HERE!

eth_tokens = tokenizer(eth_df["text"].str.cat()) 
eth_counts = Counter(ngrams(eth_tokens, n=2))
eth_counts


In [18]:
# Function token_count generates the top 10 words for a given coin
def token_count(tokens, N=3):
    """Returns the top N tokens from the frequency count"""
    return Counter(tokens).most_common(N)

In [19]:
# Use token_count to get the top 10 words for Bitcoin
# YOUR CODE HERE!

top10_btc = token_count(btc_tokens, 10)
top10_btc

In [20]:
# Use token_count to get the top 10 words for Ethereum
# YOUR CODE HERE!

top10_eth = token_count(eth_tokens, 10)
top10_eth

---

### Word Clouds

In this section, you will generate word clouds for each coin to summarize the news for each coin

In [21]:
from wordcloud import WordCloud
import matplotlib.pyplot as plt
plt.style.use('seaborn-whitegrid')
import matplotlib as mpl
mpl.rcParams['figure.figsize'] = [20.0, 10.0]

In [22]:
# Generate the Bitcoin word cloud
# YOUR CODE HERE!

btc_cloud = str(top10_btc)
wordcloud = WordCloud(colormap="RdYlBu").generate(btc_cloud)
plt.imshow(wordcloud)
plt.axis("off")
fontdict = {"fontsize": 14, "fontweight": "bold"}
plt.title("Bitcoin Top 10 Words Cloud", fontdict=fontdict)
plt.show()

In [23]:
# Generate the Ethereum word cloud
# YOUR CODE HERE!

eth_cloud = str(top10_eth)
wordcloud = WordCloud(colormap="RdYlBu").generate(eth_cloud)
plt.imshow(wordcloud)
plt.axis("off")
fontdict = {"fontsize": 14, "fontweight": "bold"}
plt.title("Ethereum Top 10 Words Cloud", fontdict=fontdict)
plt.show()

---
## 3. Named Entity Recognition

In this section, you will build a named entity recognition model for both Bitcoin and Ethereum, then visualize the tags using SpaCy.

In [24]:
import spacy
from spacy import displacy

In [25]:
# Download the language model for SpaCy
# !python -m spacy download en_core_web_sm

##JR Not sure about this - ??

In [26]:
# Load the spaCy model

nlp = spacy.load('en_core_web_sm')


---
### Bitcoin NER

In [27]:
# Concatenate all of the Bitcoin text together
# YOUR CODE HERE!

btc_ner_text = ' '.join(btc_df['text'])
btc_ner_text

In [28]:
# Run the NER processor on all of the text
# YOUR CODE HERE!

btc_docs = nlp(btc_ner_text)

# Add a title to the document
# YOUR CODE HERE!

btc_docs.user_data["title"] = "Bitcoin Named Entity Recognition (NER)"


In [29]:
# Render the visualization
# YOUR CODE HERE!

displacy.render(btc_docs, style='ent')

In [30]:
# List all Entities
# YOUR CODE HERE!

for ent in btc_docs.ents:
    print(ent.text, ent.label)

---

### Ethereum NER

In [31]:
# Concatenate all of the Ethereum text together
# YOUR CODE HERE!

eth_ner_text = ' '.join(eth_df['text'])
eth_ner_text

In [32]:
# Run the NER processor on all of the text
# YOUR CODE HERE!

eth_docs = nlp(eth_ner_text)

# Add a title to the document
# YOUR CODE HERE!

eth_docs.user_data["title"] = "Ethereum Named Entity Recognition (NER)"

In [33]:
# Render the visualization
# YOUR CODE HERE!

displacy.render(eth_docs, style='ent')

In [34]:
# List all Entities
# YOUR CODE HERE!

for ent in eth_docs.ents:
    print(ent.text, ent.label)

---